<a href="https://colab.research.google.com/github/epicskills1/forest_fire_prediction/blob/main/Forest_Fire_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install Required Packages

In [1]:
!pip install streamlit pyngrok pandas scikit-learn


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 34.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 55.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.8 MB/s eta 0:00:00


Machine Learning Code to Train the Model

In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, accuracy_score
import warnings
import pickle

warnings.filterwarnings("ignore")

# Create a CSV file with the dataset
data_csv = """
Area,Oxygen,Temperature,Humidity,Fire Occurence
Agra,40,45,20,1
Andhra Pradesh,50,30,10,1
Uttarpradesh,10,20,70,0
Bihar,60,45,70,1
Karnataka,30,48,10,1
Asansole,50,15,30,0
Bardhaman,5,35,35,0
Kolkata,5,20,70,0
America,60,32,19,1
England,30,50,45,1
Canada,40,55,80,1
Italy,30,39,100,0
Delhi,10,12,12,0
Bangladesh,20,23,23,0
Rajasthan,60,34,15,1
France,80,4,68,0
Afganishtan,25,56,79,1
Armenia,12,45,90,0
Belgium,28,43,15,1
Brazil,26,0,32,0
China,32,37,21,1
Russia,45,48,12,1
Ukrain,56,15,65,0
Turkey,67,45,33,1
Japan,78,49,8,1
Sri Lanka,89,45,18,1
Greece,21,12,44,0
Zimbabwe,11,20,59,0
Korea,1,32,70,0
Iceland,5,20,45,0
Singapore,42,43,20,1
Ecuador,32,60,30,1
Serbia,5,15,5,0
Gergoria,20,30,5,0
Israel,30,51,2,1
Korea,60,60,10,1
Sweden,10,10,60,0
Turkey,0,30,0,0
South Africa,45,45,15,1
"""

with open("Fire_Prediction_Sheet1.csv", "w") as file:
    file.write(data_csv)

# Load the data
data = pd.read_csv("Fire_Prediction_Sheet1.csv")

# Encode the categorical 'Area' column
label_encoder = LabelEncoder()
data['Area'] = label_encoder.fit_transform(data['Area'])

# Split features and target
X = data[['Oxygen', 'Temperature', 'Humidity']]
y = data['Fire Occurence']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

# Initialize and train the logistic regression model
log_reg = LogisticRegression()
log_reg.fit(X_train, y_train)

# Save the model
pickle.dump(log_reg, open('model.pkl', 'wb'))

# Evaluate the model
y_pred = log_reg.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))


Accuracy: 1.0
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00         7
           1       1.00      1.00      1.00         5

    accuracy                           1.00        12
   macro avg       1.00      1.00      1.00        12
weighted avg       1.00      1.00      1.00        12



Streamlit Application

In [2]:
%%writefile app.py
import streamlit as st
import pickle
import numpy as np

# Load the model
model = pickle.load(open('model.pkl', 'rb'))

def predict_forest(oxygen, temperature, humidity):
    # Ensure the order of inputs matches the training order: [Oxygen, Temperature, Humidity]
    input = np.array([[oxygen, temperature, humidity]]).astype(np.float64)
    prediction = model.predict_proba(input)
    pred = '{0:.{1}f}'.format(prediction[0][1], 2)  # Get the probability of fire occurrence
    return float(pred)

def main():
    st.title("Forest Fire Prediction ML App")

    html_temp = """
    <div style="background-color:#025246;padding:10px">
    <h2 style="color:white;text-align:center;">Forest Fire Prediction ML App</h2>
    </div>
    """
    st.markdown(html_temp, unsafe_allow_html=True)

    oxygen = st.text_input("Oxygen", "Type Here")
    temperature = st.text_input("Temperature", "Type Here")
    humidity = st.text_input("Humidity", "Type Here")

    safe_html = """
    <div style="background-color:#F4D03F;padding:10px">
    <h2 style="color:white;text-align:center;">Your forest is safe</h2>
    </div>
    """
    danger_html = """
    <div style="background-color:#F08080;padding:10px">
    <h2 style="color:black;text-align:center;">Your forest is in danger</h2>
    </div>
    """

    if st.button("Predict"):
        try:
            # Convert inputs to floats
            oxygen = float(oxygen)
            temperature = float(temperature)
            humidity = float(humidity)

            # Get prediction
            output = predict_forest(oxygen, temperature, humidity)
            st.success(f'The probability of fire taking place is {output}')

            if output > 0.5:
                st.markdown(danger_html, unsafe_allow_html=True)
            else:
                st.markdown(safe_html, unsafe_allow_html=True)
        except ValueError:
            st.error("Please enter valid numbers for all input fields")

if __name__ == '__main__':
    main()


Writing app.py


Streamlit App Using Ngrok

In [5]:

!pip install --upgrade pyngrok

from pyngrok import ngrok
ngrok.set_auth_token("2N13huDP2ANtbXSJQ7OQ2HvAShU_2pEtgBSQ4CM4B6mY6kTEF") # Removed extra space before this line

# Terminate all existing ngrok tunnels
ngrok.kill()

# Create a new ngrok tunnel, explicitly specifying HTTP protocol
public_url = ngrok.connect(8501, proto="http") # Specify protocol as "http"
print(f"Streamlit App URL: {public_url}")

Streamlit App URL: NgrokTunnel: "https://cf91-34-85-165-225.ngrok-free.app" -> "http://localhost:8501"


In [7]:
!pip install streamlit
import subprocess

# Run Streamlit app
subprocess.Popen(['streamlit', 'run', 'app.py'])

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 58.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 83.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.6 MB/s eta 0:00:00


<Popen: returncode: None args: ['streamlit', 'run', 'app.py']>